# Sieci Neuronowe - Laboratorium 7. Sieć Hopfielda

1. Stwórz zbiór uczący w postaci np. 4 różnych cyfr "1" i 4 cyfr "8"
2. Naucz sieć hopfielda rozpoznawać te wzorce używając metody Hebba, 
    
        hebbian_training(network, patterns)
    
3. Czy sieć zapamiętała wzorce? Uruchom ją na każdym z wzorców.
4. Dopisz kod do rysowania przykładów żeby lepiej porównywać działanie
5. Czy to jedyne wzorce zapamiętane przez sieć? Spróbuj uruchomić ją na losowych sygnałach.
6. Napisz własną metodę uczenia opartą o pseudoodwrotność
$$
W = XX^\dagger
$$
gdzie $A^\dagger$ to psuedoodwrotność Moore'a-Penrose'a, można ją również policzyć ręcznie $A^\dagger = (A^TA)^{-1}A^T$

7. Zwiększ liczbę wzorców (np. narysuj 10 różnych cyfr) i przeprowadź testy, czy któraś metoda uczenia działa lepiej?


In [1]:
"""
Kod autorstwa Matigakis Panagiotis
"""

import numpy as np
from random import randint

class InvalidWeightsException(Exception):
    pass


class InvalidNetworkInputException(Exception):
    pass


class HopfieldNetwork(object):

    def __init__(self, num_inputs):
        self._num_inputs = num_inputs
        self._weights = np.random.uniform(-1.0, 1.0, (num_inputs, num_inputs))

    def set_weights(self, weights):
        """Update the weights array"""
        if weights.shape != (self._num_inputs, self._num_inputs):
            raise InvalidWeightsException()

        self._weights = weights

    def get_weights(self):
        """Return the weights array"""
        return self._weights

    def evaluate(self, input_pattern):
        """Calculate the output of the network using the input data"""
        if input_pattern.shape != (self._num_inputs, ):
            raise InvalidNetworkInputException()

        sums = input_pattern.dot(self._weights)

        s = np.zeros(self._num_inputs)

        for i, value in enumerate(sums):
            if value > 0:
                s[i] = 1
            else:
                s[i] = -1

        return s

    def run(self, input_pattern, max_iterations=10):
        """Run the network using the input data until the output state doesn't change
        or a maximum number of iteration has been reached."""
        last_input_pattern = input_pattern

        iteration_count = 0

        while True:
            result = self.evaluate(last_input_pattern)

            iteration_count += 1

            if np.array_equal(result, last_input_pattern) or iteration_count == max_iterations:
                return result
            else:
                last_input_pattern = result
                

                
def hebbian_training(network, input_patterns):
    """Train a network using the Hebbian learning rule"""
    n = len(input_patterns)

    num_neurons = network.get_weights().shape[0]

    weights = np.zeros((num_neurons, num_neurons))

    for i in range(num_neurons):
        for j in range(num_neurons):
            if i == j:
                continue
            for m in range(n):
                weights[i, j] += input_patterns[m][i] * input_patterns[m][j]

    weights *= 1 / float(n)

    network.set_weights(weights)